In [1]:
from selenium import webdriver ## 啟動SeleniumChrom瀏覽器
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
import time
import unicodedata

### 先獲得50個股的股票代碼

In [2]:
stockidhtml = 'https://www.cmoney.tw/etf/e210.aspx'

option = webdriver.ChromeOptions()
option.add_argument('headless')
path = '/Users/jack/Desktop/Python/chromedriver' ## 指定chromedriver位置
driver = webdriver.Chrome(executable_path=path,chrome_options=option)

stockids = []

driver.get(stockidhtml)
cmoney = BeautifulSoup(driver.page_source)
table = cmoney.find('table','tb tb1')
for row in table.find_all('td',{'key':'CommKey'}):
        stockid = row.text
        stockids.append(stockid)

<ipython-input-2-de220c3bfe06>:6: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path=path,chrome_options=option)


In [3]:
stockids.remove('202103TX')
stockids.remove('202103NYF')
stockids

['2330',
 '2317',
 '2454',
 '2308',
 '2303',
 '1301',
 '1303',
 '2412',
 '2881',
 '2891',
 '3711',
 '2882',
 '3008',
 '1216',
 '2886',
 '2002',
 '2884',
 '1326',
 '3034',
 '2327',
 '2885',
 '2382',
 '2379',
 '2357',
 '5871',
 '1101',
 '2892',
 '5880',
 '2207',
 '2880',
 '3045',
 '6415',
 '2395',
 '2912',
 '4938',
 '2474',
 '2887',
 '6505',
 '2883',
 '5876',
 '2890',
 '1402',
 '2801',
 '1102',
 '4904',
 '9910',
 '2105',
 '2408',
 '6669',
 '2633']

### 爬取個股資訊

In [4]:
import requests
from datetime import datetime
import pandas as pd

In [5]:
data = {}
for stockid in stockids: 
    data[stockid] = {}
    
for stockid in stockids:
    #GET Month Sale Response
    monthSale_url = "https://marketinfo.api.cnyes.com/mi/api/v1/TWS%%3A%s%%3ASTOCK/revenue?months=30"%(stockid)
    monthSale_response = requests.get(monthSale_url)

    #營收月份
    dates_timestamp = monthSale_response.json()['data']['time']
    dates = []
    for date_timestamp in dates_timestamp:
        transform = datetime.fromtimestamp(date_timestamp)
        dates.append(transform)
    #月營收
    sale_months_json = monthSale_response.json()['data']['datasets']['saleMonth']
    sale_months = []
    for sale_month in sale_months_json:
        sale_months.append(sale_month)
    #加進Dictionary
    sale_month_dict = {'Date':dates,'Sales':sale_months}
    data[stockid]['Month_Sale'] = sale_month_dict
    
    #GET Dividend Response
    dividend_url = "https://marketinfo.api.cnyes.com/mi/api/v1/TWS%%3A%s%%3ASTOCK/divided"%(stockid)
    dividend_response = requests.get(dividend_url)
    
    #抓dividend日期 和 發放股利
    dividendDates = []
    cashDividends = []
    stockDividends = []
    for count in range(len(dividend_response.json()['data']['divides'])):
        dividendDate = dividend_response.json()['data']['divides'][count]['formatDate']
        dividendCash = dividend_response.json()['data']['divides'][count]['formatCashDividend']
        dividendStock = dividend_response.json()['data']['divides'][count]['formatStockDividend']
        dividendDates.append(dividendDate)
        cashDividends.append(dividendCash)
        stockDividends.append(dividendStock)
    dividend_dict = {'Date':dividendDates,'Cash':cashDividends,'Stock':stockDividends}
    data[stockid]['Dividends'] = dividend_dict
    
    #Get Profile Response
    profile_url = "https://marketinfo.api.cnyes.com/mi/api/v1/TWS:%s:STOCK/info"%(stockid)
    profile_response = requests.get(profile_url)
    brandCH = profile_response.json()['data']['companyName']
    brandEN = profile_response.json()['data']['companyEnglishName']
    industryType = profile_response.json()['data']['industryType']
    description = profile_response.json()['data']['description']
    profile_dict = {'BrandCH':brandCH,'BrandEN':brandEN,'IndustryType':industryType,'Description':description}
    data[stockid]['Profile'] = profile_dict
    
    #Get EPS
    eps_url = "https://marketinfo.api.cnyes.com/mi/api/v1/TWS%%3A%s%%3ASTOCK/eps"%(stockid)
    eps_response = requests.get(eps_url)
    quarter_eps = eps_response.json()['data']['quarter']['eps']
    quarter_date_timestamps = eps_response.json()['data']['quarter']['time']
    quarter_dates = []
    for quarter_date_timestamp in quarter_date_timestamps:
        transform = datetime.fromtimestamp(quarter_date_timestamp)
        quarter_dates.append(transform)
    eps_dict = {'EPS_Date':quarter_dates,'Quarter_EPS':quarter_eps}
    data[stockid]['EPS'] = eps_dict

    #Get Finance Data
    finance_url = "https://marketinfo.api.cnyes.com/mi/api/v1/statement/TWS%%3A%s%%3ASTOCK/ratio/quarter"%(stockid)
    finance_response = requests.get(finance_url)

    finance_dict = {}

    items = finance_response.json()['data']['datas']
    for item in range(len(items)):
        if item != 6:
            name = finance_response.json()['data']['datas'][item]['name'] #Example: 營業毛利率
            finance_dict[name] = {}
            name_data = []
            for finance_data in finance_response.json()['data']['datas'][item]['datasets']:
                name_data.append(finance_data['percent'])
                finance_dict[name]['Data'] = name_data
            finance_range = finance_response.json()['data']['range'] #List of finance range ex:2020Q3
            finance_dict[name]['Date'] = finance_range
    data[stockid]['Finance'] = finance_dict

In [6]:
data

{'2330': {'Month_Sale': {'Date': [datetime.datetime(2018, 6, 1, 8, 0),
    datetime.datetime(2018, 7, 1, 8, 0),
    datetime.datetime(2018, 8, 1, 8, 0),
    datetime.datetime(2018, 9, 1, 8, 0),
    datetime.datetime(2018, 11, 1, 8, 0),
    datetime.datetime(2018, 12, 1, 8, 0),
    datetime.datetime(2019, 1, 1, 8, 0),
    datetime.datetime(2019, 3, 1, 8, 0),
    datetime.datetime(2019, 4, 1, 8, 0),
    datetime.datetime(2019, 5, 1, 8, 0),
    datetime.datetime(2019, 6, 1, 8, 0),
    datetime.datetime(2019, 7, 1, 8, 0),
    datetime.datetime(2019, 8, 1, 8, 0),
    datetime.datetime(2019, 9, 1, 8, 0),
    datetime.datetime(2019, 10, 1, 8, 0),
    datetime.datetime(2019, 11, 1, 8, 0),
    datetime.datetime(2019, 12, 1, 8, 0),
    datetime.datetime(2020, 1, 1, 8, 0),
    datetime.datetime(2020, 2, 1, 8, 0),
    datetime.datetime(2020, 3, 1, 8, 0),
    datetime.datetime(2020, 4, 1, 8, 0),
    datetime.datetime(2020, 5, 1, 8, 0),
    datetime.datetime(2020, 6, 1, 8, 0),
    datetime.datetime(

In [7]:
import _pickle as pickle
with open('tw0050Data', 'wb') as file:
     file.write(pickle.dumps(data))

In [8]:
tw0050 = pickle.load(open( "tw0050Data", "rb" ) )

In [9]:
s2330 = pd.DataFrame(tw0050['2330']['Month_Sale'])
s2330

,Date,Sales
0,2018-06-01 08:00:00,70438298
1,2018-07-01 08:00:00,74370924
2,2018-08-01 08:00:00,91055038
3,2018-09-01 08:00:00,94921920
4,2018-11-01 08:00:00,98389414
5,2018-12-01 08:00:00,89830598
6,2019-01-01 08:00:00,78093827
7,2019-03-01 08:00:00,79721587
8,2019-04-01 08:00:00,74693615
9,2019-05-01 08:00:00,80436931
